In [124]:
from sqlalchemy import create_engine
import pandas as pd
from fuzzywuzzy import process, fuzz
from geopy.geocoders import Nominatim

In [125]:
connection_url =  "secret"

In [126]:
dataframe = pd.read_sql_table("locations", connection_url, "public")

In [127]:
dataframe.head()

,id,type,agency,program_type,name,address1,address2,address3,city,state,...,contact_last_name,date_first_record,date_recent_record,notes,phone,fax,url,email,created_at,updated_at
0,4,ICE ERO Field Office,ICE,None,Atlanta Field Office,180 Ted Turner Dr. SW Suite 522,None,None,Atlanta,GA,...,None,2016-07-26,2016-07-29,None,(404) 893-1210,None,https://www.ice.gov/contact/ero#wcm-survey-tar...,Atlanta.Outreach@ice.dhs.gov,2018-07-10 19:44:58.934400+00:00,2018-07-11 06:46:46.973700+00:00
1,5,ICE ERO Field Office,ICE,None,SND Field Office,None,None,None,None,None,...,None,2015-12-22,2017-01-22,None,None,None,https://www.ice.gov/contact/ero#wcm-survey-tar...,None,2018-07-10 19:44:58.934400+00:00,2018-07-11 06:46:47.152500+00:00
2,6,ICE ERO Field Office,ICE,None,San Diego Field Office,880 Front Street #2232,None,None,San Diego,CA,...,None,2015-08-08,2016-09-26,None,(619) 436-0410,None,https://www.ice.gov/contact/ero#wcm-survey-tar...,SanDiego.Outreach@ice.dhs.gov,2018-07-10 19:44:58.934400+00:00,2018-07-11 06:46:47.333600+00:00
3,7,ICE ERO Field Office,ICE,None,Newark Field Office,970 Broad St. 11th Floor,None,None,Newark,NJ,...,None,2016-02-17,2016-05-06,None,(973) 645-3666,None,https://www.ice.gov/contact/ero#wcm-survey-tar...,Newark.Outreach@ice.dhs.gov,2018-07-10 19:44:58.934400+00:00,2018-07-11 06:46:47.514800+00:00
4,8,ICE ERO Field Office,ICE,None,San Antonio Field Office,1777 NE Loop 410 Floor 15,None,None,San Antonio,TX,...,None,2015-07-26,2016-04-26,None,(210) 283-4750,None,https://www.ice.gov/contact/ero#wcm-survey-tar...,SanAntonio.Outreach@ice.dhs.gov,2018-07-10 19:44:58.934400+00:00,2018-07-11 06:46:47.689000+00:00


In [128]:
trac_locations = pd.read_pickle('trac_locations.pickle')

In [129]:
trac_locations.head(20)

,Departures,Facility Name,City,State,Facility Type,Per Diem Rate
0,199,Abbott Home,New York City,New York,Juvenile Facility,Not Available
1,85,Abilene Hold Room,Abilene,Kansas,Holding/Staging Facility,Not Available
2,1816,Adelanto Detention Facility,Adelanto,California,Intergovernmental Service Agreement,"1455 (GM); $111.92 (1-1455), $42.36 (1456-1940)"
3,6,Agana Immigration Holding Facility,Hagatna,Guam,Holding/Staging Facility,Not Available
4,2,Alamosa Hold Room,Alamosa,Colorado,Holding/Staging Facility,Not Available
5,86,Albany County Jail,Albany,New York,Intergovernmental Service Agreement,$119.30
6,21,Albuquerque Hold Room,Albuquerque,New Mexico,Holding/Staging Facility,Not Available
7,3,Alexandria City Jail,Alexandria,Virginia,Intergovernmental Service Agreement,Not Available
8,12043,Alexandria Staging Facility,Alexandria,Louisiana,Holding/Staging Facility,Not Available
9,114,Alice Peterson Residence,Mesa,Arizona,Juvenile Facility,Not Available


In [130]:
choices = trac_locations['Facility Name']
j = 0
k = 0
geolocator = Nominatim(user_agent="iceicedata")
dataframe['longitude'] = None
dataframe['latitude'] = None
for i in range(0, len(dataframe)):
    name = dataframe.iloc[i]['name']
    match = process.extractOne(name, choices, scorer=fuzz.token_set_ratio)
    if match[1]>80:
        city = trac_locations.at[match[2], 'City']
        state = trac_locations.at[match[2], 'State']
        location = geolocator.geocode(city+', '+state)
        dataframe.at[i,'city'] = city
        dataframe.at[i,'state'] = state
        if location is not None:
            dataframe.at[i,'longitude'] = location.longitude
            dataframe.at[i, 'latitude'] = location.latitude
        else:
            print(city+', '+state+' has no location')
        print(name, match)
        j += 1
    else:
        print('****', name, match)
        k += 1
print(j,' matches and ', k, ' fails.')

Atlanta Field Office ('Atlanta District Office Holding Room', 82, 15)
SND Field Office ('Dallas Field Office Holding Facility', 86, 119)
**** San Diego Field Office ('Seattle Field Office Holding Room', 73, 400)
**** Newark Field Office ('Dallas Field Office Holding Facility', 77, 119)
**** San Antonio Field Office ('Dallas Field Office Holding Facility', 67, 119)
Phoenix Field Office ('Phoenix District Office Holding Facility', 82, 349)
**** Los Angeles Field Office ('Dallas Field Office Holding Facility', 67, 119)
ABRAXAS ACADEMY DETENTION CENTER ('Haile Detention. Center', 84, 185)
ADELANTO ICE PROCESSING CENTER ('El Paso Service Processing Center', 83, 145)
ARTESIA FAMILY RESIDENTIAL CENTER ('Artesia Family Residential Center', 100, 14)
ATLANTA DISTRICT HOLD ROOM ('Atlanta District Office Holding Room', 89, 15)
AUSTIN DRO HOLD ROOM ('Austin DRO Hold Room', 100, 18)
**** AZ REM OP COORD CENTER (AROCC) ('Arizona Removal Operations Coordination Center (AROCC)', 70, 13)
BALTIMORE DD&P 

**** BCFS Camp Delta ('Berks County Family Shelter', 48, 33)
**** BCFS Camp Echo ('Board of Child Care', 48, 39)
**** BCFS Extended Care ('Crittenton Foster Care', 50, 115)
**** BCFS Fairfield ('Fairfax County Jail', 55, 155)
**** BCFS Raymondville ('Grayson County Jail', 50, 178)
**** BCFS San Antonio LTFC ('MVM Transport -San Antonio', 69, 309)
**** BCFS San Antonio Staff Secure ('MVM Transport -San Antonio', 61, 309)
Bethany Christian Services Shelter ('Bethany Christian Services, Grand Rapids', 87, 36)
Bethany Christian Services TFC ('Bethany Christian Services, Grand Rapids', 93, 36)
**** Bethany LIRS ("Bethany Children's Home", 74, 35)
**** Bethany USCCB ("Bethany Children's Home", 70, 35)
Board of Child Care Shelter ('Board of Child Care', 100, 39)
Board of Child Care TFC ('Board of Child Care', 100, 39)
**** Bokenkamp TFC ('Rapp Sec Center', 43, 371)
Brazoria County Youth Homes ('Brazoria County Youth Homes', 100, 45)
**** Camp Bruceville ('Louisville Hold Room', 46, 264)
Cardi

Youth for Tomorrow LTFC ('Youth for Tomorrow', 100, 511)
A NEW LEAF ('New Leaf', 100, 316)
BAPTIST CHILD & FAMILY SERVICES ('Baptist Child & Family Services', 100, 23)
BAPTIST CHILD & FAMILY SVCS-SNA ('Baptist Child & Family Services-SNA', 94, 28)
**** BCFS Chavaneaux ('Case Guadalupe', 41, 67)
**** BCFS Harlingen ('Harlingen Hold Room', 78, 189)
BCFS San Antonio ('MVM Transport -San Antonio', 81, 309)
**** BCFS San Antonio TFC ('MVM Transport -San Antonio', 71, 309)
**** BCFS SNA ('Baptist Child & Family Services-SNA', 55, 28)
**** BCFS TFC ('Baptist Child & Family Services-TFC', 55, 29)
BETHANY C.S.,GRAND RAPIDS ('Bethany Christian Services, Grand Rapids', 91, 36)
BETHANY CHRISTIAN SERVICES OF MD ('Bethany Christian Services, Grand Rapids', 90, 36)
**** BokenKamp ('Krome/Miami Hub', 42, 240)
CAYUGA CENTERS ('Cayuga Centers', 100, 80)
CHILDRENS VILLAGE ("Children's Village", 97, 96)
Children's Village ("Children's Village", 100, 96)
CRITTENDEN SERVICES ('Crittenton Services', 89, 116)

**** Individual Sponsor in NC ('El Paso Boeing Drive Hold Room', 48, 141)
**** Individual Sponsor in ND ('El Paso Boeing Drive Hold Room', 52, 141)
**** Individual Sponsor in OH ('El Paso Boeing Drive Hold Room', 52, 141)
**** Individual Sponsor in OK ('El Paso Boeing Drive Hold Room', 52, 141)
**** Individual Sponsor in OR ('El Paso Boeing Drive Hold Room', 52, 141)
**** Individual Sponsor in PA ('El Paso Boeing Drive Hold Room', 48, 141)
**** Individual Sponsor in RI ('El Paso Boeing Drive Hold Room', 48, 141)
**** Individual Sponsor in CT ('David L. Moss Criminal Justice Center (Jail)', 49, 121)
**** Individual Sponsor in DE ('Indianapolis Hold Room', 48, 213)
**** Individual Sponsor in DC ('Indianapolis Hold Room', 48, 213)
**** Individual Sponsor in IN ('El Paso Boeing Drive Hold Room', 47, 141)
**** Individual Sponsor in IA ('El Paso Boeing Drive Hold Room', 44, 141)
**** Individual Sponsor in KS ('El Paso Boeing Drive Hold Room', 48, 141)
**** Individual Sponsor in KY ('El Paso 

In [131]:
dataframe.head(20)

,id,type,agency,program_type,name,address1,address2,address3,city,state,...,date_recent_record,notes,phone,fax,url,email,created_at,updated_at,longitude,latitude
0,4,ICE ERO Field Office,ICE,None,Atlanta Field Office,180 Ted Turner Dr. SW Suite 522,None,None,Atlanta,Georgia,...,2016-07-29,None,(404) 893-1210,None,https://www.ice.gov/contact/ero#wcm-survey-tar...,Atlanta.Outreach@ice.dhs.gov,2018-07-10 19:44:58.934400+00:00,2018-07-11 06:46:46.973700+00:00,-84.3902,33.7491
1,5,ICE ERO Field Office,ICE,None,SND Field Office,None,None,None,Dallas,Texas,...,2017-01-22,None,None,None,https://www.ice.gov/contact/ero#wcm-survey-tar...,None,2018-07-10 19:44:58.934400+00:00,2018-07-11 06:46:47.152500+00:00,-96.7969,32.7763
2,6,ICE ERO Field Office,ICE,None,San Diego Field Office,880 Front Street #2232,None,None,San Diego,CA,...,2016-09-26,None,(619) 436-0410,None,https://www.ice.gov/contact/ero#wcm-survey-tar...,SanDiego.Outreach@ice.dhs.gov,2018-07-10 19:44:58.934400+00:00,2018-07-11 06:46:47.333600+00:00,None,None
3,7,ICE ERO Field Office,ICE,None,Newark Field Office,970 Broad St. 11th Floor,None,None,Newark,NJ,...,2016-05-06,None,(973) 645-3666,None,https://www.ice.gov/contact/ero#wcm-survey-tar...,Newark.Outreach@ice.dhs.gov,2018-07-10 19:44:58.934400+00:00,2018-07-11 06:46:47.514800+00:00,None,None
4,8,ICE ERO Field Office,ICE,None,San Antonio Field Office,1777 NE Loop 410 Floor 15,None,None,San Antonio,TX,...,2016-04-26,None,(210) 283-4750,None,https://www.ice.gov/contact/ero#wcm-survey-tar...,SanAntonio.Outreach@ice.dhs.gov,2018-07-10 19:44:58.934400+00:00,2018-07-11 06:46:47.689000+00:00,None,None
5,9,ICE ERO Field Office,ICE,None,Phoenix Field Office,2035 N. Central Avenue,None,None,Phoenix,Arizona,...,2016-01-06,None,(602) 766-7030,None,https://www.ice.gov/contact/ero#wcm-survey-tar...,Phoenix.Outreach@ice.dhs.gov,2018-07-10 19:44:58.934400+00:00,2018-07-11 06:46:47.861700+00:00,-112.077,33.4486
6,10,ICE ERO Field Office,ICE,None,Los Angeles Field Office,300 North Los Angeles St. Room 7631,None,None,Los Angeles,CA,...,2016-09-26,None,(213) 830-7911,None,https://www.ice.gov/contact/ero#wcm-survey-tar...,LosAngeles.Outreach@ice.dhs.gov,2018-07-10 19:44:58.934400+00:00,2018-07-11 06:46:48.029500+00:00,None,None
7,12,ICE or Shared Facility,None,JUVENILE,ABRAXAS ACADEMY DETENTION CENTER,None,None,None,Caldwell,Idaho,...,2018-04-20,None,None,None,None,None,2018-07-10 19:44:58.934400+00:00,2018-07-11 06:46:48.199900+00:00,-116.689,43.6679
8,13,ICE or Shared Facility,None,DIGSA,ADELANTO ICE PROCESSING CENTER,None,None,None,El Paso,Texas,...,2018-04-19,None,None,None,None,None,2018-07-10 19:44:58.934400+00:00,2018-07-11 06:46:48.379300+00:00,-106.487,31.76
9,14,ICE or Shared Facility,ICE,FAMILY,ARTESIA FAMILY RESIDENTIAL CENTER,None,None,None,Artesia,New Mexico,...,2014-12-18,None,None,None,None,None,2018-07-10 19:44:58.934400+00:00,2018-07-11 06:46:48.547000+00:00,-104.403,32.8423
